In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

tqdm.pandas()
from data_processing.bridge import *
from PyEMD import CEEMDAN, Visualisation

In [ ]:
combined = load_data()

In [ ]:
combined

In [ ]:
trange = combined.index.max() - combined.index.min()

## CEEMD

In [ ]:
# TODO - original code interpolates over NaNs in input data, check if we need to do that, or PyEMD is doing it

In [ ]:
# noise
ns = np.arange(.1, .5, .05)
ns = [.25]

# number of ensembles
NR = 100

In [ ]:
# TODO - unclear whether noise should be epsilon, or noise scale
ceemd = CEEMDAN(trials=NR, epsilon=ns[0], processes=8, parallel=True)
imfs = ceemd.ceemdan(combined['y'].to_numpy(), combined.index.to_numpy(), progress=True)

In [ ]:
assert np.isnan(imfs).sum() == 0, 'NaNs in imfs'

In [ ]:
imfs.shape

In [ ]:
# Visualisation - expected output
t = combined.index
s = combined['y']

plt.plot(t, s)
plt.title('Shoreline vs. time')
plt.show()

In [ ]:
imfs.shape

In [ ]:
freq = np.fft.rfftfreq(len(imfs[8,:]), d=1)

In [ ]:
x = np.fft.rfft(imfs[2,:])
i = np.argmax(abs(x))
print(i, freq[i])

In [ ]:
plt.plot(abs(x))

In [ ]:
from data_processing.significance import zero_crossings, maxima

In [ ]:
# Approximate signal frequency - maxima per year
for i in range(imfs.shape[0]):
    print(i, maxima(imfs[i, :]), 365 * maxima(imfs[i,:]) / trange)

In [ ]:
plt.plot(imfs[7,:])

In [ ]:
Visualisation(ceemd).plot_imfs(imfs, t=combined.index.to_numpy())

In [ ]:
Visualisation(ceemd).plot_instant_freq(range(len(t)))